# Reading a fasta file

In [1]:
def loadFasta(filename):
    
    if(filename.endswith(".gz")):
        fp=zip.open(filename,'r')
    else:
        fp=open(filename,'r')
    data=fp.read().split('>')
    fp.close()
    
    data.pop(0)
    headers=[]
    sequences=[]
    for sequence in data:
        lines=sequence.split('\n')
        headers.append(lines.pop(0))
        sequences.append(''.join(lines))
    return(headers, sequences)
header,seq=loadFasta("VC.fa")

Selecting the size of the dna string

In [2]:
dna_string=seq[0][:20000]

In [3]:
len(dna_string)

20000

Displaying the DNA string

In [4]:
dna_string

'...'

# Functions for the Burrow wheeler's transform

In [5]:
# function to append dollar
def dollar(dna_string,required=True):
    if required:
        dna_string+="$"
    else:
        dna_string=dna_string[:-1]
    return dna_string

# creating bwm from string
def word_list(dna_string):
    word_lst=[dna_string[i:]+dna_string[:i] for i in range(len(dna_string))] 
    return sorted(word_lst)# sort the full table

# getting the bwt from bwm
def bwt_transform(word_list):
    bwt=""
    for seq in word_list:
        bwt+=seq[-1] # add last letter from every row in table
    return bwt
# a function that combines all the functions above for easy user experience
def bwt(dna_string):
    return bwt_transform(word_list(dollar(dna_string,required=True)))

# Functions for ranking the strings in the bwm

In [6]:
def rankBwt(bw):
    """ Given BWT string bw, returns a parallel list of B-ranks. Also
    returns tots, a mapping from characters to # times the
    character appears in BWT. """
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots:
            tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots
def firstCol(tots):
    """ Return a map from characters to the range of cells in the first
    column containing the character. """
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

# A very fast Inverse BWT 

Using only first and last column we retrieve the original burrow wheeler's transform

In [7]:
def invBwt(bw):
    """ Make T from BWT(T) """
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    rowi = 0
    t = "$"
    while bw[rowi] != "$":
        c = bw[rowi]
        t = c + t
        rowi = first[c][0] + ranks[rowi]
    return t

Displaying the Burrow Wheel Transform of the string given

In [8]:
bwt_transfm=bwt(dna_string)
bwt_transfm

'...'

# Compression

## Functions for compression

The encoding and decoding is for memory optimisation and is one of the methods for compressing the string. The compression type being used is run length encoding 

In [9]:
def encode(message): 
    encoded_message = "" 
    i = 0
   
    while (i <= len(message)-1): 
        count = 1
        ch = message[i] 
        j = i 
        while (j < len(message)-1): 
            if (message[j] == message[j+1]): 
                count = count+1
                j = j+1
            else: 
                break
        if count>1:
            encoded_message=encoded_message+str(count)+ch
        else:
            encoded_message=encoded_message+ch
        i = j+1
    return encoded_message

Displaying the encoded version of the transform

In [10]:
encoded_string=encode(bwt_transfm)
encoded_string

'3CT3CAGC3GTGAGCGAGTAGA2TCTACTA3T3ACTAC5ATCTGC2TGCG2CACGC2TGAT2C2ATAC2ACTCAG3C3ACACT4ACG2CGT3GAGCG2CGC2A7CGT2CATCTA2GTATACG3C2TGT2C2ACACTC2A3C2ATC2TGTACGATGAC2A2CACGATACACTAGTGTAGCATCAGTG2CG4C2A2TC2AG2ACTCTGACTA2CACGC2ACACGTA2CTCAC3A3CGCT2ACTCGATA2TG2CACATCG2CGAGCGATCT2GCACTC3TG4CAGC2GCGCAGACTA2T2C2AG2TCTA2CT2ATA2T2GCGCTGT3C3GT2CATAGCTCT5CGC2ACAG2CAT2GACGCG2TGT2A2GAG2TC4A2TC2AGCGC3AGCTACATG2ATACTGAT2ATCGC3AT2AGA2T2ATAT2A4TCGTG2A2TCGCGCACTA3TGC2ACACAC2AG2CACT3GACGCGCTC3A2CG2CACTC3ATAG2ACA2G2CA3CG2AT2AT2G3AC2A2G2CATA2CTCTC3AGAGCG2CGATA2GTCA3C2ATCA2C2AC6AGTCG2AC2TA2TACAT2A3CTAG3T2ACG5A2CGATCAG2ACTCACA2CATGTACA2GATA2TACAGT3ATA2TCATACTA2C6TC2G2C3ATC2GACT4AGACTACAGT2C2TGCGC2T4ACTAGCATA2CT2AT2ACACAGTC2TATGAC3AG5A2GAG2ATACA2CAT2ACGA2CA2G2CTCAGAGTCGAGATGA2GA3GA3T2AC3AT2CGAGA2CAC2ATGCGT2AT2ATC2A2GATC2G2ACACT2CGT3C2G3CGACTC2AT2AC2AG5ACGCGAGCAGACAC2G3ATACT2ACATCGAT2AG2TACAC5TGTACT2C5ATCG2CGCGAG2ACATG2A2TC2T2ACTCAG2CTAG2CAC2G2CGCG4C4A2C2ATGCATACGTCGCA2CT3G2TACA2C2GTCTA2CTAC3TA2CACA2TCG4CAGACTACATCA

In [11]:
def decode(encoded_string):
    encoded_list=list(encoded_string)
    for i in range(len(encoded_list)-1):
        if encoded_list[i].isdigit():
            encoded_list[i]=encoded_list[i+1]*(int(encoded_list[i])-1)
    return ''.join(encoded_list)

Decoding and displaying the retrieved transform string

In [12]:
decoded_string=decode(encoded_string)
decoded_string

'...'

Retrieving the original string and displaying it

In [13]:
inverseBwt=dollar(invBwt(decoded_string),required=False)
inverseBwt

'...'

In [14]:
dna_string==inverseBwt

True

# Pattern Searching

Creating a list of patterns

In [15]:
from itertools import permutations
patterns=[''.join(i) for i in permutations('AGCT')]
len(patterns)

24

## Function for pattern matching

Along with telling if pattern is present in the string or not, it also tells the number of occurences in the transformed string

In [16]:
def countMatches(bw, p):
    """ Given BWT(T) and a pattern string p, return the number of times
    p occurs in T. """
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    l, r = first[p[-1]]
    i = len(p)-2
    while i >= 0 and r > l:
        c = p[i]
        # scan from left, looking for occurrences of c
        j = l
        while j < r:
            if bw[j] == c:
                l = first[c][0] + ranks[j]
                break
            j += 1
        if j == r:
            l = r
            break # no occurrences -> no match
        r -= 1
        while bw[r] != c:
            r -= 1
        r = first[c][0] + ranks[r] + 1
        i -= 1
    return r - l

A list to count the occurences

In [17]:
check=[countMatches(bwt_transfm,i) for i in patterns]

Displaying the patterns and its occurences in the Burrow Wheel Transform of the string

In [22]:
for i in range(len(check)):
    print("occurences of %2s are: %s" % (patterns[i],check[i]))

occurences of AGCT are: 81
occurences of AGTC are: 45
occurences of ACGT are: 39
occurences of ACTG are: 78
occurences of ATGC are: 91
occurences of ATCG are: 106
occurences of GACT are: 49
occurences of GATC are: 82
occurences of GCAT are: 109
occurences of GCTA are: 64
occurences of GTAC are: 46
occurences of GTCA are: 68
occurences of CAGT are: 84
occurences of CATG are: 75
occurences of CGAT are: 92
occurences of CGTA are: 41
occurences of CTAG are: 23
occurences of CTGA are: 91
occurences of TAGC are: 58
occurences of TACG are: 56
occurences of TGAC are: 72
occurences of TGCA are: 96
occurences of TCAG are: 99
occurences of TCGA are: 56
